Материалы для работы:
* https://habr.com/ru/articles/768562/
* https://huggingface.co/learn/audio-course/ru/chapter5/asr_models

# Преимущества решения

1. Простая реализация.
2. За счет large модели, можно работать с разными языками.
3. Качественное распознавание речи в аудио.
4. Большой потенциал для дообучения модели (в завимисмотси от контекста решаемой нами задачи) -> ускоренное обучение и экономия ресурсов.

# Недостатки решения
1. Тесты проводились на хорошо предобработанных и чистых данных, что очень редко бывает в условиях продакшина. Необходимо в зависимости от контекста задачи добавить этап с предобработкой данных (убрать шумы, выделить голоса, изменить формат кодировки и тд). Не смотря на то, что сама whisper модель обучена на большом массиве разнообразных аудио, необходимо держать во внимании указанный недостаток. 


# Решение задачи для аудио на Английском

In [2]:
import torch
from transformers import pipeline

# Скачиваем модель для распознавания аудио
# Medium model работает только с английским языком, если необходимо использовать другой язык, то нужно скачать large модель
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-medium", device=device
)

In [3]:
import os
import torchaudio
from torchaudio.datasets import LIBRISPEECH
import shutil
import warnings
import difflib

warnings.filterwarnings("ignore")

In [4]:
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

#  Скачиваем 1 инстанс датасета для теста нашей модели
try:
    dataset = LIBRISPEECH(data_dir, url="test-clean", download=True)
except Exception as e:
    print(f"Ошибка: {e}")
    exit(1)

sample = dataset[0]
waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id = sample

# Определяем директорию для сохранения аудиофайла
output_dir = "../audio"
os.makedirs(output_dir, exist_ok=True)

# Сохраняем аудиофайл в формате WAV
torchaudio.save(os.path.join(output_dir, "test_audio.wav"), waveform, sample_rate)

# Сохраняем текстовый файл с транскрипцией аудиофайла
with open(os.path.join(output_dir, "test_audio.txt"), "w") as f:
    f.write(utterance)

# Удаляем директорию с данными для сохранения памяти
shutil.rmtree(data_dir)
print("Загрузка данных завершена")

100%|██████████| 331M/331M [00:44<00:00, 7.80MB/s] 


Test audio WAV file and corresponding text file created successfully.


In [5]:
warnings.filterwarnings("ignore")

# Делаем транскрипцию аудиофайла
result = pipe('../audio/test_audio.wav', max_new_tokens=256)

# Выполняем сравнение транскрипции с оригинальным текстом
transcribed_text = result["text"].lower()
print(f"Транскрипция: {transcribed_text}")

with open('../audio/test_audio.txt') as file:
    original_text = file.read().lower()
print(f"Оригинальный текст: {original_text}")

# Подсчитываем коэффициент сходства
similarity_ratio = difflib.SequenceMatcher(None, transcribed_text, original_text).ratio()

print("Коэффициет сходства:", round(similarity_ratio*100,3), "%")

Транскрипция:  he hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton-pieces to be ladled out in thick, peppered, flour-fattened sauce.
Оригинальный текст: he hoped there would be stew for dinner turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick peppered flour fattened sauce
Коэффициет сходства: 97.196 %


# Решение задачи для аудио на Русском языке

In [1]:
import torch
from transformers import pipeline

# Скачиваем модель для распознавания аудио
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-large", device=device
)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [5]:
warnings.filterwarnings("ignore")

# Делаем транскрипцию аудиофайла
result = pipe('../audio/test_audion_rus.wav', max_new_tokens=256)

# Выполняем сравнение транскрипции с оригинальным текстом
transcribed_text = result["text"].lower()
print(f"Транскрипция: {transcribed_text}")

with open('../audio/test_audio_rus.txt') as file:
    original_text = file.read().lower()
print(f"Оригинальный текст: {original_text}")

# Подсчитываем коэффициент сходства
similarity_ratio = difflib.SequenceMatcher(None, transcribed_text, original_text).ratio()

print("Коэффициет сходства:", round(similarity_ratio*100,3), "%")

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Транскрипция:  привет, мир! это тестовое аудио для проверки корректности распознавания речи большой языковой моделью.
Оригинальный текст: привет мир это тестовое аудио для проверки корректности распознавания речи большой языковой моделью
Коэффициет сходства: 98.02 %
